In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import json
from tqdm.notebook import tqdm
import random
import numpy as np
import pandas as pd
from torch.optim.lr_scheduler import ExponentialLR

c:\Users\Vitaliy\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [4]:
def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [2]:
df = pd.read_excel('concat_df.xlsx', index_col=0)

In [5]:
df

,0,1
0,Какие конкретные проекты по строительству и ре...,Благодарим вас за интерес к развитию дорожной ...
1,Каковы планы по расширению сети общественного ...,Благодарим вас за ваш вопрос относительно план...
2,Как муниципалитет собирается справляться с про...,Благодарим вас за ваш вопрос относительно проб...
3,Как осуществляется мониторинг качества обществ...,Спасибо за ваш вопрос относительно мониторинга...
4,Как решаются вопросы связанные с пересадками м...,Благодарим вас за ваш вопрос относительно вопр...
...,...,...
145,Есть ли программы улучшения детских площадок и...,Благодарим вас за вопрос о программе улучшения...
146,Существуют ли проекты по сохранению и восстано...,Благодарим вас за интересный вопрос о проектах...
147,Каким образом решаются вопросы утилизации отхо...,Благодарим вас за важный вопрос о системе утил...
148,Как обеспечивается безопасность детей на пешех...,Благодарим вас за вопрос о безопасности детей ...


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [7]:
model

T5ForConditionalGeneration(
  (shared): Embedding(20100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(20100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [8]:
pairs = [[x, y] for x, y in zip(df[0].to_list(), df[1].to_list())]

In [9]:
pairs[0]

['Какие конкретные проекты по строительству и реконструкции дорог предусмотрены в ближайших планах для нашего района?',
 'Благодарим вас за интерес к развитию дорожной инфраструктуры в нашем районе. Мы ценим ваш вопрос относительно конкретных проектов по строительству и реконструкции дорог в ближайших планах. Муниципальные органы постоянно работают над улучшением дорожной сети нашего района. В настоящее время ведется активная разработка проектов, включающих в себя строительство новых дорог, реконструкцию существующих и другие мероприятия по повышению качества транспортной инфраструктуры. Мы уверены, что обеспечение безопасности и комфорта движения наших жителей является приоритетом, и ваше мнение играет важную роль в этом процессе. Мы призываем вас активно участвовать в обсуждении и предоставлять свои предложения, так как именно ваш взгляд может стать ценным вкладом в развитие нашего района. Благодарим вас за активное участие в жизни нашего района и за вопрос, который важен для нас все

In [12]:
from tqdm import tqdm

In [14]:
batch_size = 16
report_steps = 200
epochs = 1000

model.train()
losses = []
for epoch in tqdm(range(epochs)):
    random.shuffle(pairs)
    for i in tqdm(range(0, int(len(pairs) / batch_size)), leave=False):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        
    if epoch < 300:
        scheduler.step()






  0%|          | 0/1000 [01:16<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), f'rut5_{epochs}.pt')

In [56]:
model.load_state_dict(torch.load(f'rut5_{epochs}.pt'))

<All keys matched successfully>

In [57]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [58]:
answer('Планируется ли постройка много этажных гаражей, снос старых?', max_new_tokens=10000)

'Установленные гаражи будет произведены в летний период 2023г. Строительство гаражей имеет для строительства и в эксплуатацию на территории муниципальных земельных участков, расположенных на территории муниципальных земельных участков, расположенных на территории муниципальных земельных участках, расположенных на территории муниципальных земельных участках, расположенных на территории муниципальных земельных участках, расположенных на территории муниципальных земельных участках. Это будет произведено в летний период 2023г.'

In [15]:
data = json.load(open('RuBQ_2.0_test.json', encoding='utf-8'))

FileNotFoundError: [Errno 2] No such file or directory: 'RuBQ_2.0_test.json'

In [69]:
texts = [x['question_text'] for x in data]